In [1]:
import os

from astrometry.util.fits import fits_table
from astrometry.util.util import wcs_pv2sip_hdr

import fitsio
import numpy as np
import pylab as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table

import sep
import tractor

from tractor import NullWCS
from tractor.galaxy import *
from tractor.sersic import *


In [2]:
class Grid(object):
    def __init__(self,ramin,ramax,decmin,decmax,pixscale):
        self.npix_ra= int(deg_to_arcsec(ramax-ramin)/pixscale + 1)
        self.npix_dec= int(deg_to_arcsec(decmax-decmin)/pixscale + 1)
        self.ra_centers= np.linspace(ramin,ramax,num=self.npix_ra)
        self.dec_centers= np.linspace(decmin,decmax,num=self.npix_dec)
    def nearest_cell_index(self,ra,dec):
        '''return x,y index for img of nearest ra,dec pixel center to ra,dec'''
        return np.argsort(np.abs(self.ra_centers-ra))[0], np.argsort(np.abs(self.dec_centers-dec))[0]


In [3]:
#open fits files
num = '26'
prefix = 'redadd_'+num+'_HSC-I_full_'
path = os.path.dirname(os.path.abspath(num))+'/hsc_images/'+num+'/'+prefix

img_data = fits.open(path+'img.fits')[0].data[400:800,400:800]
psf_data = fits.open(path+'psf.fits')[0].data[400:800,400:800]
sig_data = fits.open(path+'sig.fits')[0].data[400:800,400:800]
bad_data = fits.open(path+'bad.fits')[0].data[400:800,400:800]

obj_cat = Table.read(os.path.dirname(os.path.abspath(num))+'/'+num+'_tractor_ready/tractor_'+num+'.fits')

In [4]:
invvar_data = (1.0 / (sig_data ** 2.0))
invvar_data[bad_data > 0] = np.nan

w = NullWCS(pixscale=0.168)

In [5]:
tim = tractor.Image(data=img_data,
            invvar=invvar_data,
            psf=psf_data,
            wcs=w,
            sky=tractor.ConstantSky(0),
            photocal=tractor.LinearPhotoCal(tractor.NanoMaggies.zeropointToScale(27),
                                    band='r')
            )

In [6]:
obj_cat

x,y,x2,y2,xy,a,b,theta,flux,point_source
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
308.754997344,2.06837412783,2.85085962201,1.92293960705,-0.161521907025,1.6965174675,1.37681794167,-0.167507335544,15.1361637115,0.0
289.247901505,3.87949484684,2.83404700123,3.97577324759,-0.645741058597,2.06562519073,1.59468281269,-1.14736270905,10.7689361572,0.0
257.117552803,9.52864303905,2.11646897117,2.34192221063,-0.137008617232,1.55132770538,1.43240129948,-1.12963414192,27.6240844727,1.0
23.5529669723,15.2519187282,1.85771139504,1.97976705337,-0.401082785986,1.52461087704,1.2300568819,-0.860897898674,7.81157064438,0.0
398.611095041,22.9991457042,0.237657891863,1.21648399534,-0.00512421581424,1.10295546055,0.487474173307,-1.56556141376,2.05110788345,0.0
167.926590939,21.8721129849,3.49037195343,2.98972577006,-0.103674500068,1.87376403809,1.72310936451,-0.196327850223,21.7597293854,1.0
185.915946864,20.7756128205,17.593632825,12.8584390743,6.92780741183,4.74839353561,2.81155323982,0.620743811131,507.236328125,0.0
242.457483833,28.3989617067,2.8119161754,2.82453598643,0.0102029987016,1.68232655525,1.67518043518,1.06233084202,64.6484832764,1.0
314.416024617,32.626254065,1.02601409314,0.405556830417,-0.073678581115,1.01717424393,0.630021929741,-0.116588801146,1.7537099123,0.0


In [20]:
sources=[]
for i in range(len(obj_cat)):
    x = obj_cat['x'][i]
    y = obj_cat['y'][i]
    flux = obj_cat['flux'][i]
    # if the object is a point source, add it as such
    if obj_cat['point_source'][i] == 1.0:
        sources.append( tractor.PointSource(tractor.PixPos(x,y), tractor.Flux(flux)) )
    else:
        sources.append( tractor.ExpGalaxy(tractor.PixPos(x,y),tractor.Flux(flux)))

In [21]:
print(len(sources))

57


In [23]:
tractor = Tractor([tim], sources)

mod = tractor.getModelImage(0,minsb=0.,srcs=sources)

plt.clf()
plt.imshow(np.log(mod + noisesigma),
           interpolation='nearest', origin='lower', cmap='gray')
plt.title('Galaxies')

AttributeError: 'Flux' object has no attribute 'getFlux'